### 区别于DNN，CNN中的$a^l$，$z^l$，$\delta^l$不再是向量，而是一个三维的张量。
## 第一步，已知池化层误差$\delta^l$，推导上一层卷积层误差$\delta^{l-1}$。
   首先，把$\delta^l$的所有子矩阵大小还原成池化之前的大小，然后，如果是MAX，则把$\delta^{l-1}$所有子矩阵的各个池化局域的值还原到做前向传播算法得到最大值的位置。如果是Average，则把$\delta^{l-1}$的所有子矩阵的各个池化局域的值取平均后放在还原后的子矩阵位置。这个过程称为upsample。   
   举例说明，设我们的池化区域大小是2x2。$\delta^l$的第k个子矩阵为:
   \begin{eqnarray}
   \delta_k^l = \left( \begin{array}{ccc} 2& 8 \\ 4& 6 \end{array} \right)
   \end{eqnarray}    
   先讲$\delta_k^l$做还原，即变成：
    \begin{eqnarray}
   \left( \begin{array}{ccc} 0&0&0&0 \\ 0&2& 8&0 \\ 0&4&6&0 \\ 0&0&0&0 \end{array} \right)
    \end{eqnarray}   
    如果是MAX，假设我们之前在前向传播时记录的最大值位置分别是左上，右下，右上，左下，则转换后的矩阵为：
     \begin{eqnarray}
    \left( \begin{array}{ccc} 2&0&0&0 \\ 0&0& 0&8 \\ 0&4&0&0 \\ 0&0&6&0 \end{array} \right)
    \end{eqnarray}   
    如果是Average，则进行平均：转换后的矩阵为：
    \begin{eqnarray}
    \left( \begin{array}{ccc} 0.5&0.5&2&2 \\ 0.5&0.5&2&2 \\ 1&1&1.5&1.5 \\ 1&1&1.5&1.5 \end{array} \right)
     \end{eqnarray}   
### 这样我们就可以得到$\delta_k^{l-1}$：
\begin{eqnarray}
\delta_k^{l-1} = \frac{\partial J(W,b)}{\partial a_k^{l-1}} \frac{\partial  a_k^{l-1}}{\partial z_k^{l-1}} = upsample(\delta_k^l) \odot \sigma^{'}(z_k^{l-1})
\end{eqnarray}
### 对于张量$\delta^{l-1}$，有：
\begin{eqnarray}
\delta^{l-1} =  upsample(\delta^l) \odot \sigma^{'}(z^{l-1})
\end{eqnarray}     


## 第二步，已知卷积层的$\delta^l$，推导上一隐藏层的$\delta^{l-1}$。
### 根据前向传播公式：
\begin{eqnarray}
a^l= \sigma(z^l) = \sigma(a^{l-1}*W^l +b^l)
\end{eqnarray} 
### 以及DNN中求得的递推公式：
\begin{eqnarray}
\delta^{l} = \frac{\partial J(W,b)}{\partial z^l} = \frac{\partial J(W,b)}{\partial z^{l+1}}\frac{\partial z^{l+1}}{\partial z^{l}} = \delta^{l+1}\frac{\partial z^{l+1}}{\partial z^{l}}
\end{eqnarray} 
### 可求得：
\begin{eqnarray}
\delta^{l-1} =  \delta^{l}\frac{\partial z^{l}}{\partial z^{l-1}} = \delta^{l}*rot180(W^{l}) \odot  \sigma^{'}(z^{l-1})
\end{eqnarray} 
### 式子中$rot180()$表示卷积核被旋转了180度，即上下翻转一次，接着左右翻转一次。    
### 举例说明：  
假设我们l−1层的输出$a^{l-1}$是一个3x3矩阵，第l层的卷积核$w^{l}$是一个2x2矩阵，采用1像素的步幅，则输出$z^{l}$是一个2x2的矩阵。简化$b^{l}$都是0,则有：
\begin{eqnarray}
a^{l-1}*W^l = z^{l}
\end{eqnarray} 

矩阵a,W,z分别为：
\begin{eqnarray}
\left( \begin{array}{ccc} a_{11}&a_{12}&a_{13} \\ a_{21}&a_{22}&a_{23}\\ a_{31}&a_{32}&a_{33} \end{array} \right)   * \left( \begin{array}{ccc} w_{11}&w_{12}\\ w_{21}&w_{22} \end{array} \right) = \left( \begin{array}{ccc} z_{11}&z_{12}\\ z_{21}&z_{22} \end{array} \right)
\end{eqnarray} 
根据卷积的定义，可得：
\begin{eqnarray}
&&z_{11} = a_{11}w_{11} + a_{12}w_{12} + a_{21}w_{21} +   a_{22}w_{22}\\
&&z_{12} = a_{12}w_{11} + a_{13}w_{12} + a_{22}w_{21} +   a_{23}w_{22}\\
&&z_{21} = a_{21}w_{11} + a_{22}w_{12} + a_{31}w_{21} +   a_{32}w_{22}\\
&&z_{22} = a_{22}w_{11} + a_{23}w_{12} + a_{32}w_{21} +   a_{33}w_{22}
\end{eqnarray} 
求梯度误差$\nabla a^{l-1}$:
\begin{eqnarray}
\nabla a^{l-1} = \frac{\partial J(W,b)}{\partial a^{l-1}} = \frac{\partial J(W,b)}{\partial z^{l}} \frac{\partial z^{l}}{\partial a^{l-1}} = \delta^{l} \frac{\partial z^{l}}{\partial a^{l-1}}
\end{eqnarray} 
假设z矩阵对应的反向传播误差是$\delta_{11}, \delta_{12}, \delta_{21}, \delta_{22}$组成的2x2矩阵,则利用上面梯度的式子和4个等式，可以分别写出$\nabla a^{l-1}$的9个标量的梯度:   
\begin{eqnarray}
&&\nabla a_{11} = \delta_{11}w_{11}\\
&&\nabla a_{12} = \delta_{11}w_{12} + \delta_{12}w_{11}\\
&&\nabla a_{13} = \delta_{12}w_{12}\\
&&\nabla a_{21} = \delta_{11}w_{21} + \delta_{21}w_{11}\\
&&\nabla a_{22} = \delta_{11}w_{22} + \delta_{12}w_{21} + \delta_{21}w_{12} + \delta_{22}w_{11}\\
&&\nabla a_{23} = \delta_{12}w_{22} + \delta_{22}w_{12}\\
&&\nabla a_{31} = \delta_{21}w_{21}\\
&&\nabla a_{32} = \delta_{21}w_{22} + \delta_{22}w_{21}\\
&&\nabla a_{33} = \delta_{22}w_{22}\\
\end{eqnarray} 
上面9个式子可用矩阵卷积的形式表示，即：
\begin{eqnarray}
\left( \begin{array}{ccc} 0&0&0&0 \\ 0&\delta_{11}& \delta_{12}&0 \\ 0&\delta_{21}&\delta_{22}&0 \\ 0&0&0&0 \end{array} \right) * \left( \begin{array}{ccc} w_{22}&w_{21}\\ w_{12}&w_{11} \end{array} \right)  = \left( \begin{array}{ccc} \nabla a_{11}&\nabla a_{12}&\nabla a_{13} \\ \nabla a_{21}&\nabla a_{22}&\nabla a_{23}\\ \nabla a_{31}&\nabla a_{32}&\nabla a_{33} \end{array} \right)
\end{eqnarray} 
可以看出卷积核被翻转了180度。

## 第三部，已知每一层的梯度误差$\delta^l$，推导各层的W,b的梯度。
### 全连接层可按DNN的BP算法求得，池化层无W，b。所以只需要求出卷积层的W，b的梯度。
### 由：
\begin{eqnarray}
z^l = a^{l-1}*W^l +b
\end{eqnarray} 
### 有：
\begin{eqnarray}
\frac{\partial J(W,b)}{\partial W^{l}} = \frac{\partial J(W,b)}{\partial z^{l}}\frac{\partial z^{l}}{\partial W^{l}} =a^{l-1} *\delta^l
\end{eqnarray} 
 此处沿用第二部简化的例子说明。对于第l层，某个卷积核矩阵W的导数可以表示如下： 
\begin{eqnarray}
\frac{\partial J(W,b)}{\partial W_{pq}^{l}} = \sum\limits_i\sum\limits_j(\delta_{ij}^lx_{i+p-1,j+q-1}^{l-1})
\end{eqnarray}      
假设输入a是4x4的矩阵，卷积核W是3x3的矩阵，输出z是2x2的矩阵,那么反向传播的z的梯度误差δ也是2x2的矩阵。   
根据上面式子，有：    
\begin{eqnarray}
&&\frac{\partial J(W,b)}{\partial W_{11}^{l}} = a_{11}\delta_{11} + a_{12}\delta_{12} + a_{21}\delta_{21} +  a_{22}\delta_{22}\\
&&\frac{\partial J(W,b)}{\partial W_{12}^{l}} = a_{12}\delta_{11} + a_{13}\delta_{12} + a_{22}\delta_{21} +  a_{23}\delta_{22}\\
&&\frac{\partial J(W,b)}{\partial W_{13}^{l}} = a_{13}\delta_{11} + a_{14}\delta_{12} + a_{23}\delta_{21} +  a_{24}\delta_{22}\\
&&\frac{\partial J(W,b)}{\partial W_{21}^{l}} = a_{21}\delta_{11} + a_{22}\delta_{12} + a_{31}\delta_{21} +  a_{32}\delta_{22}\\
&& ......
\end{eqnarray}    
整理成矩阵形式为：
\begin{eqnarray}
\frac{\partial J(W,b)}{\partial W^{l}} =\left( \begin{array}{ccc} a_{11}&a_{12}&a_{13}&a_{14} \\ a_{21}&a_{22}&a_{23}&a_{24} \\ a_{31}&a_{32}&a_{33}&a_{34} \\ a_{41}&a_{42}&a_{43}&a_{44} \end{array} \right) * \left( \begin{array}{ccc} \delta_{11}& \delta_{12} \\ \delta_{21}&\delta_{22} \end{array} \right)
\end{eqnarray} 

### 对于b的梯度求解，由于是一个三维的张量，不能直接等于$\delta^l$。通常的做法是将$\delta^l$的各个子矩阵的项分别求和：
\begin{eqnarray}
\frac{\partial J(W,b)}{\partial b^{l}} = \sum\limits_{u,v}(\delta^l)_{u,v}
\end{eqnarray} 

#### <font color=red>附注：上面各式中的*代表卷积操作，非矩阵的乘法。<font>